In [3]:
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_cluster import knn_graph
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt
import numpy as np

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import tensorflow as tf
import pandas as pd
#from source import Dataset as ds

2024-02-12 14:52:35.873897: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 14:52:35.873954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 14:52:35.875686: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-12 14:52:35.887460: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-12 14:52:37.067158: W tensorflow/compiler/tf2

In [4]:
a = torch.Tensor(np.arange(10))
b = torch.Tensor(np.arange(10,20))

In [5]:
a = torch.reshape(a, (2,5))
b = torch.reshape(b, (2,5))

In [6]:
torch.cat((a,b), dim = 1)

tensor([[ 0.,  1.,  2.,  3.,  4., 10., 11., 12., 13., 14.],
        [ 5.,  6.,  7.,  8.,  9., 15., 16., 17., 18., 19.]])

In [7]:
#read data
path = '/beegfs/desy/group/unihh2/cms-llp/run3/v10/ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV/dataframes/'
alldf = []
def process_file(i):
    store = pd.HDFStore(path + 'output_' + str(i) + ".h5")
    dfnew = store.select("df",start=0,stop=-1)
    store.close()
    return dfnew


In [8]:
frames = [process_file(j) for j in range(10)]

In [9]:
branches_rechits = [
"cscClusteredRechitsEta",
"cscClusteredRechitsPhi",
"cscClusteredRechitsX",
"cscClusteredRechitsY",
"cscClusteredRechitsZ",
"cscClusteredRechitsTpeak",
"cscClusteredRechitsTwire",
"cscClusteredRechitsChamber",
"cscClusteredRechitsStation",
"cscClusteredRechitsIndex",]

energy_key = "cscRechitCluster_match_gLLP_e"

In [10]:
# Check GPU availability

print("Num GPUs available: ", len(tf.config.list_physical_devices('GPU')), 
      "\nCuda available: ", torch.cuda.is_available(),
     "\nCuda device count: ", torch.cuda.device_count(),
     "\ncurrent device: ", torch.cuda.current_device(),
     "\ndevice info: ", torch.cuda.device(0), 
      "\ndevice name: ", torch.cuda.get_device_name(0))

Num GPUs available:  2 
Cuda available:  True 
Cuda device count:  2 
current device:  0 
device info:  <torch.cuda.device object at 0x2af1f4903070> 
device name:  Tesla P100-SXM2-16GB


In [62]:
# Iterate through all clusters
cluster_features = ["cscRechitClusterX",
                    "cscRechitClusterY",
                    "cscRechitClusterZ",
                    "cscRechitClusterTimeWeighted",
                    "cscRechitClusterTimeSpreadWeightedAll",
                    "cscRechitClusternXY",
                    "cscRechitClusternZ",
                    "cscRechitClusterXSpread",
                    "cscRechitClusterYSpread",
                    "cscRechitClusterZSpread",
                    "cscRechitClusterXYSpread",
                    "cscRechitClusterRSpread",
                    "cscRechitClusterEtaPhiSpread",
                    "cscRechitClusterEtaSpread",
                    "cscRechitClusterPhiSpread",
                    "cscRechitClusterDeltaRSpread",
                    "cscRechitClusterMajorAxis",
                    "cscRechitClusterMinorAxis",
                    "cscRechitClusterSkewX",
                    "cscRechitClusterSkewY",
                    "cscRechitClusterSkewZ",
                    "cscRechitClusterKurtX",
                    "cscRechitClusterKurtY",
                    "cscRechitClusterKurtZ"]
                    #"cscRechitClusterPhi",
                    #"cscRechitClusterEta"]

                    
def make_graphs(df_signal):
    num_edges = 8
    node_features = []
    graph_features = []
    edges = []
    label_e = []
    for i in range(df_signal.index[-1][0]):
        # graph level
        graph_level = []
        for feature in cluster_features:
            graph_level.append(df_signal[feature][i][0])
            
        # features for edge making
        nodes_edge = torch.Tensor(np.transpose([df_signal['cscClusteredRechitsRelEta'][i].to_numpy(),
                                                df_signal['cscClusteredRechitsRelPhi'][i].to_numpy()]))
        # features for nodes
        nodes = torch.Tensor(np.transpose([df_signal['cscClusteredRechitsRelEta'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsRelPhi'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsRelX'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsRelY'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsRelZ'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsTpeak'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsTwire'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsStation'][i].to_numpy(),
                                           df_signal['cscClusteredRechitsChamber'][i].to_numpy()]))
        
        graph_features.append(torch.Tensor(graph_level))
        node_features.append(nodes)
        edges.append(knn_graph(nodes_edge, num_edges))
        label_e.append(torch.Tensor([df_signal[energy_key][i][0]]))

    print(len(node_features), len(edges), len(label_e), len(graph_features))
    graph_data = []
    for i in range(len(node_features)):
        graph_data.append(Data(x=node_features[i], edge_index=edges[i], u=graph_features[i], y=label_e[i]))
    return graph_data

def make_glevel(df_signal):
    graph_features = []
    label_e = []
    df = pd.DataFrame()
    for feature in cluster_features:
        df[feature] = []
    df[energy_key] = []
    for i in range(df_signal.index[-1][0]):
        row = []
        for feature in cluster_features:
            row.append(df_signal[feature][i][0])
        row.append(df_signal[energy_key][i][0])
        df.loc[i] = row
    print('done')
    return df

In [63]:
test = [make_glevel(frame) for frame in frames]
result = pd.concat(test)

done
done
done
done
done
done
done
done
done
done


In [51]:
result.to_hdf('gcndata_glvl.h5', 'df', format='table')

In [24]:
torch.save(glevel, '/beegfs/desy/group/unihh2/cms-llp/run3/v10/ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV/graphs/graphs9k8nn_' + str(1) + 'ONLYgLvl' + '.pt')

In [65]:
# make list of graphs in Data format
out_data = []
for frame in frames:
    out_data.extend(make_graphs(frame))

983 983 983 983
967 967 967 967
958 958 958 958
951 951 951 951
936 936 936 936
901 901 901 901
962 962 962 962
1033 1033 1033 1033
962 962 962 962
933 933 933 933


In [66]:
out_data[-1].u.size()

torch.Size([24])

In [60]:
result[cluster_features]

,cscRechitClusterX,cscRechitClusterY,cscRechitClusterZ,cscRechitClusterTimeWeighted,cscRechitClusterTimeSpreadWeightedAll,cscRechitClusterTime,cscRechitClusterTimeSpread,cscRechitClusternXY,cscRechitClusternZ,cscRechitClusterXSpread,...,cscRechitClusterMajorAxis,cscRechitClusterMinorAxis,cscRechitClusterSkewX,cscRechitClusterSkewY,cscRechitClusterSkewZ,cscRechitClusterKurtX,cscRechitClusterKurtY,cscRechitClusterKurtZ,cscRechitClusterPhi,cscRechitClusterEta
0,-138.806732,632.686279,-840.816711,5.114727,6.585391,5.738067,6.535855,55.0,55.0,22.584471,...,0.040976,0.032285,-2.650515,-14.028798,0.060668,-1.058820,-1.460573,-1.470452,1.786767,-1.077286
1,163.848969,166.184586,736.253906,-33.162510,12.568985,-34.912071,12.562634,57.0,57.0,45.193497,...,0.071866,0.066804,30.732649,8.675519,61.625153,-0.934861,-1.187875,-1.459484,0.792475,1.866298
2,-368.993042,-214.768661,-690.334412,-1.066650,9.462804,-0.566933,9.456563,254.0,254.0,28.449711,...,0.042103,0.032417,-79.328110,-20.227047,-119.588188,9.346846,1.471736,18.136015,-2.614484,-1.257916
3,-209.355896,-386.587738,921.330383,0.602103,9.488232,2.044143,9.488521,123.0,123.0,20.567471,...,0.069454,0.030705,-13.153602,-10.078119,8.788766,-0.183109,-1.371722,-0.560612,-2.067127,1.485618
4,162.949951,-392.275299,1011.986267,-0.566438,10.701680,1.840356,10.890076,380.0,380.0,22.671556,...,0.072712,0.042096,6.453502,-5.511337,-40.607410,-0.162686,-0.799589,-0.463900,-1.177088,1.602653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,32.796932,416.011932,859.548889,0.373818,16.038273,1.535494,16.095537,444.0,444.0,21.175175,...,0.080820,0.050393,-2.248130,2.812188,70.219795,-0.174925,-1.090080,0.072446,1.492123,1.470052
929,417.158264,175.900513,-925.281616,-0.481041,12.199575,1.949075,12.568431,276.0,276.0,35.137573,...,0.050800,0.030692,16.019102,18.185633,-11.512364,-0.168678,0.792737,-1.536924,0.399041,-1.463051
930,128.887344,197.772827,-994.805481,-0.272097,13.439157,3.330842,13.747252,1589.0,1589.0,26.148706,...,0.128002,0.100586,14.425070,24.234791,22.177952,0.780371,0.880210,-1.483981,0.993231,-2.145382
931,414.434998,-484.236633,-1010.239380,0.091606,10.294052,0.609839,10.339939,542.0,542.0,25.840275,...,0.057013,0.027877,-0.791690,-0.495154,34.668972,-0.892329,-1.062649,-1.293403,-0.862915,-1.241012


In [67]:
# Save finished graphs here

torch.save(out_data, '/beegfs/desy/group/unihh2/cms-llp/run3/v10/ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV/graphs/graphs9k8nn_' + str(1) + '.pt')


In [28]:
a = out_data[0].u
b = out_data[1].u

In [32]:
c = None
torch.cat((a,b), 0)

tensor([-1.3881e+02,  6.3269e+02, -8.4082e+02,  5.1147e+00,  6.5854e+00,
         5.7381e+00,  6.5359e+00,  5.5000e+01,  5.5000e+01,  2.2584e+01,
         3.1850e+01,  1.3738e-01,  1.3890e+01,  3.2774e+01,  1.1482e-02,
         4.0626e-02,  3.2724e-02,  5.2166e-02,  4.0976e-02,  3.2285e-02,
        -2.6505e+00, -1.4029e+01,  6.0668e-02, -1.0588e+00, -1.4606e+00,
        -1.4705e+00,  1.7868e+00, -1.0773e+00,  1.6385e+02,  1.6618e+02,
         7.3625e+02, -3.3163e+01,  1.2569e+01, -3.4912e+01,  1.2563e+01,
         5.7000e+01,  5.7000e+01,  4.5193e+01,  3.1421e+01,  1.5978e+02,
         3.4995e+01,  5.2464e+01,  1.8093e-02,  6.8463e-02,  7.0287e-02,
         9.8120e-02,  7.1866e-02,  6.6804e-02,  3.0733e+01,  8.6755e+00,
         6.1625e+01, -9.3486e-01, -1.1879e+00, -1.4595e+00,  7.9247e-01,
         1.8663e+00])

In [33]:
a

tensor([-1.3881e+02,  6.3269e+02, -8.4082e+02,  5.1147e+00,  6.5854e+00,
         5.7381e+00,  6.5359e+00,  5.5000e+01,  5.5000e+01,  2.2584e+01,
         3.1850e+01,  1.3738e-01,  1.3890e+01,  3.2774e+01,  1.1482e-02,
         4.0626e-02,  3.2724e-02,  5.2166e-02,  4.0976e-02,  3.2285e-02,
        -2.6505e+00, -1.4029e+01,  6.0668e-02, -1.0588e+00, -1.4606e+00,
        -1.4705e+00,  1.7868e+00, -1.0773e+00])

In [34]:
b

tensor([ 1.6385e+02,  1.6618e+02,  7.3625e+02, -3.3163e+01,  1.2569e+01,
        -3.4912e+01,  1.2563e+01,  5.7000e+01,  5.7000e+01,  4.5193e+01,
         3.1421e+01,  1.5978e+02,  3.4995e+01,  5.2464e+01,  1.8093e-02,
         6.8463e-02,  7.0287e-02,  9.8120e-02,  7.1866e-02,  6.6804e-02,
         3.0733e+01,  8.6755e+00,  6.1625e+01, -9.3486e-01, -1.1879e+00,
        -1.4595e+00,  7.9247e-01,  1.8663e+00])